# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler


# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [2]:
# Memuat dataset
file_path = "clustered_garis_kemiskinan.csv"
df = pd.read_csv(file_path)

# Menampilkan informasi awal tentang dataset
df.info(), df.head()

# Pastikan kolom fitur dan target ada dalam dataset
print("Kolom dalam dataset:", df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5224 entries, 0 to 5223
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   provinsi  5224 non-null   object 
 1   jenis     5224 non-null   object 
 2   daerah    5224 non-null   object 
 3   tahun     5224 non-null   float64
 4   periode   5224 non-null   object 
 5   gk        5224 non-null   float64
 6   cluster   5224 non-null   int64  
 7   Cluster   5224 non-null   int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 326.6+ KB
Kolom dalam dataset: Index(['provinsi', 'jenis', 'daerah', 'tahun', 'periode', 'gk', 'cluster',
       'Cluster'],
      dtype='object')


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [3]:


# Memilih fitur dan target
X = df[['gk', 'tahun']]  # Sesuaikan nama kolom jika perlu
y = df['Cluster']  # Pastikan ini adalah kolom target

# Membagi dataset menjadi training (80%) dan testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Melakukan normalisasi fitur numerik menggunakan StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Pastikan hasil split berhasil
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"y_train shape: {y_train.shape}")

X_train_scaled shape: (4179, 2)
y_train shape: (4179,)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [4]:
# Inisialisasi model
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5)
}

# Melatih dan mengevaluasi model awal
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)  # Melatih model
    y_pred = model.predict(X_test)  # Prediksi
    accuracy = accuracy_score(y_test, y_pred)  # Menghitung akurasi
    results[name] = accuracy

# Menampilkan hasil akurasi awal
results

{'Logistic Regression': 0.3751196172248804,
 'Decision Tree': 0.3894736842105263,
 'Random Forest': 0.384688995215311,
 'K-Nearest Neighbors': 0.34832535885167465}



**terlihat Logistic regression, decicions tree, dan random forest mendapat nilai 1.0.**\
dimana itu artinya musngkin terdapat overfitting.\
Jadi saya memilih KNN untu dituning lagi.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [5]:
# Evaluasi setiap model lebih lanjut
evaluation_results = {}
confusion_matrices = {}

for name, model in models.items():
    y_pred = model.predict(X_test)  # Prediksi data uji
    
    # Menghitung metrik evaluasi
    report = classification_report(y_test, y_pred, output_dict=True)
    evaluation_results[name] = report
    
    # Menghitung confusion matrix
    confusion_matrices[name] = confusion_matrix(y_test, y_pred)

# Menampilkan hasil evaluasi model
evaluation_results

c:\Users\Reina_Chi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Reina_Chi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Reina_Chi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

{'Logistic Regression': {'0': {'precision': 0.3668763102725367,
   'recall': 0.49575070821529743,
   'f1-score': 0.42168674698795183,
   'support': 353.0},
  '1': {'precision': 0.38204225352112675,
   'recall': 0.6027777777777777,
   'f1-score': 0.4676724137931034,
   'support': 360.0},
  '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 332.0},
  'accuracy': 0.3751196172248804,
  'macro avg': {'precision': 0.2496395212645545,
   'recall': 0.3661761619976917,
   'f1-score': 0.29645305359368507,
   'support': 1045.0},
  'weighted avg': {'precision': 0.2555431088935991,
   'recall': 0.3751196172248804,
   'f1-score': 0.3035574073227409,
   'support': 1045.0}},
 'Decision Tree': {'0': {'precision': 0.38186813186813184,
   'recall': 0.3937677053824363,
   'f1-score': 0.3877266387726639,
   'support': 353.0},
  '1': {'precision': 0.3988439306358382,
   'recall': 0.38333333333333336,
   'f1-score': 0.3909348441926346,
   'support': 360.0},
  '2': {'precision': 0.388059701492

In [6]:
# Menyiapkan data untuk tabel evaluasi
model_names = list(evaluation_results.keys())
accuracy_scores = [results[name] for name in model_names]

precision_scores = []
recall_scores = []
f1_scores = []

for name in model_names:
    precision_scores.append([evaluation_results[name][str(i)]['precision'] if str(i) in evaluation_results[name] else 0 for i in range(3)])
    recall_scores.append([evaluation_results[name][str(i)]['recall'] if str(i) in evaluation_results[name] else 0 for i in range(3)])
    f1_scores.append([evaluation_results[name][str(i)]['f1-score'] if str(i) in evaluation_results[name] else 0 for i in range(3)])

# Membuat DataFrame untuk hasil evaluasi
eval_df = pd.DataFrame({
    "Model": model_names,
    "Accuracy": accuracy_scores,
    "Precision (Kelas 0)": [p[0] for p in precision_scores],
    "Precision (Kelas 1)": [p[1] for p in precision_scores],
    "Precision (Kelas 2)": [p[2] for p in precision_scores],
    "Recall (Kelas 0)": [r[0] for r in recall_scores],
    "Recall (Kelas 1)": [r[1] for r in recall_scores],
    "Recall (Kelas 2)": [r[2] for r in recall_scores],
    "F1-Score (Kelas 0)": [f[0] for f in f1_scores],
    "F1-Score (Kelas 1)": [f[1] for f in f1_scores],
    "F1-Score (Kelas 2)": [f[2] for f in f1_scores],
})

# Menampilkan tabel hasil evaluasi
eval_df

,Model,Accuracy,Precision (Kelas 0),Precision (Kelas 1),Precision (Kelas 2),Recall (Kelas 0),Recall (Kelas 1),Recall (Kelas 2),F1-Score (Kelas 0),F1-Score (Kelas 1),F1-Score (Kelas 2)
0,Logistic Regression,0.375120,0.366876,0.382042,0.000000,0.495751,0.602778,0.000000,0.421687,0.467672,0.000000
1,Decision Tree,0.389474,0.381868,0.398844,0.388060,0.393768,0.383333,0.391566,0.387727,0.390935,0.389805
2,Random Forest,0.384689,0.377465,0.387812,0.389058,0.379603,0.388889,0.385542,0.378531,0.388350,0.387292
3,K-Nearest Neighbors,0.348325,0.347737,0.370607,0.321138,0.478754,0.322222,0.237952,0.402861,0.344725,0.273356


Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [7]:
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"Unique labels in y_train: {set(y_train)}")

X_train_scaled shape: (4179, 2)
y_train shape: (4179,)
Unique labels in y_train: {0, 1, 2}


In [8]:
# Hyperparameter tuning untuk K-Nearest Neighbors (KNN)
knn_params = {
    "n_neighbors": [3, 5, 7, 9],
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "manhattan"],
}

grid_knn = GridSearchCV(KNeighborsClassifier(), knn_params, cv=3, scoring="accuracy", n_jobs=-1, error_score="raise")
grid_knn.fit(X_train_scaled, y_train)

# Menampilkan parameter terbaik untuk KNN
print("Best KNN Parameters:", grid_knn.best_params_)

Best KNN Parameters: {'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'uniform'}


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [9]:
# Menggunakan model KNN dengan hyperparameter terbaik
best_knn = KNeighborsClassifier(metric='euclidean', n_neighbors=7, weights='uniform')
best_knn.fit(X_train_scaled, y_train)

# Prediksi pada data uji
y_pred_knn = best_knn.predict(X_test_scaled)

# Evaluasi performa model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Menghitung metrik evaluasi
accuracy = accuracy_score(y_test, y_pred_knn)
print(f"Accuracy setelah tuning: {accuracy:.4f}")

# Menampilkan classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_knn))

# Menampilkan Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_knn))

Accuracy setelah tuning: 0.4478

Classification Report:
              precision    recall  f1-score   support

           0       0.43      0.52      0.47       353
           1       0.48      0.47      0.47       360
           2       0.43      0.34      0.38       332

    accuracy                           0.45      1045
   macro avg       0.45      0.45      0.44      1045
weighted avg       0.45      0.45      0.44      1045


Confusion Matrix:
[[184  85  84]
 [125 170  65]
 [116 102 114]]


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Analisis Hasil Evaluasi Model KNN
Berdasarkan hasil evaluasi, berikut analisisnya:

🔹 Akurasi setelah tuning: 44.78% → Model belum cukup baik untuk klasifikasi.
🔹 Precision, Recall, dan F1-Score rendah untuk semua kelas (~0.43 - 0.48).
🔹 Confusion Matrix menunjukkan banyak kesalahan prediksi antar kelas.

 **rekomendasi** tahapannya.
1. Perbandingan Hasil Evaluasi Sebelum dan Setelah Tuning
- Akurasi sebelum tuning: 43.92%

- Akurasi setelah tuning: 44.78% (peningkatan 0.86%)

- Precision sebelum tuning: 0.43 - 0.48

- Precision setelah tuning: 0.43 - 0.48 (tidak ada perubahan signifikan)

- Recall sebelum tuning: 0.33 - 0.52

- Recall setelah tuning: 0.34 - 0.52 (sedikit meningkat)

- F1-Score sebelum tuning: 0.38 - 0.47

- F1-Score setelah tuning: 0.38 - 0.47 (tidak ada perubahan signifikan)

2. Identifikasi kelemahan model, seperti:\
   **Precision atau Recall rendah untuk kelas tertentu**
   - Precision atau Recall rendah untuk kelas tertentu.
   - Recall untuk kelas 2 hanya 34%, yang berarti model sering salah mengklasifikasikan instance dari kelas ini.
   - Precision pada semua kelas masih rendah (~0.43 - 0.48), menunjukkan model sering salah memprediksi.

   **Overfitting atau Underfitting?**
   - Model kemungkinan underfitting, karena performa tetap rendah meskipun sudah dilakukan tuning.
   - KNN mungkin tidak optimal untuk dataset ini, terutama jika ada banyak fitur yang kurang relevan atau terdapat noise.

3. Rekomendasi Tindakan Lanjutan
- Coba algoritma lain
- Random Forest atau SVM bisa menjadi pilihan karena lebih fleksibel dalam menangani dataset dengan pola kompleks.
- Coba XGBoost, yang sering memberikan hasil lebih baik pada klasifikasi.
-  Feature Engineering
- Lakukan Feature Selection untuk mengurangi noise dan meningkatkan akurasi.
- Uji kembali dengan memilih fitur yang lebih informatif.
- Penanganan Data
- Jika data tidak seimbang, coba metode Oversampling (SMOTE) atau Undersampling untuk memperbaiki distribusi kelas.
- Cek kembali preprocessing seperti normalisasi atau scaling agar fitur memiliki skala yang seragam.
\\
